#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017

#### PROJECT

# SCUPER: Social Circle and User PErsonality based Recommendation System

### Team : Aniket Bonde, Nagaraj Janakiraman, Sudheer Dantuluri

### [25% of final grade]

### Due: Tuesday, May 2 by 11:59pm

*Goal of this project:* Get hand-on experience with building a recommendation system.


# 1. Introduction and Problem Statement

        Recommender systems play a pivotal role in boosting the sales of major e-commerce systems. With rapid increase in the number of registered users and products on e-commerce systems, the problem of cold start for users (new users into the RS with little historical behavior) and the sparsity of datasets (the proportion of rated user-item pairs in all the user-item pairs of RS) has become increasingly intractable.

        Fortunately, the appearance of web2.0 greatly improved user’s initiative on the Internet, and then brought volume of social networks such as Facebook, Twitter, Yelp, Douban, Epinions, etc. This new factor of social network - the interpersonal interest based on circles of friends brings opportunities to the recommender system to solve the cold start & sparsity problem of datasets.

        In this Project, we propose to implement a personalized recommendation model that fuses in user’s personal interest and social factors like interpersonal similarity and interpersonal influence that is based on social network. The factor of user personal interest captures direct connection between user and item latent feature vectors, and social factors capture connections between user and his/here friend's latent feature vectors.

# 2. Related Work

        Recently, there have been some works that address to solve the cold start problem by factoring in social network features. Some recent significant contributions are CircleCon Model [1], ContextMF model [2] and [3]. [1] proposed a model that uses the concept of ‘inferred trust circle’ based on the domain-obvious circles of friends on social network. However, [2] demonstrated that individual preference (personal interest) is also a significant factor in social network. [3] builds on these ideas and proposes a personalized recommendation model based on probabilistic matrix factorization that considers three factors - personal interest, interpersonal interest similarity and interpersonal influence. Detailed explanation for each factor is presented in the methodology section.

        In our project, we desire to understand the effects of factors proposed in [3], and evaluate it on a real dataset like ‘Yelp’ to recommend businesses for users with very less rating history (cold start scenario). We would evaluate the model that considers user personal interest and social circle and compare it against a model like classic Matrix Factorization that neglects factors from social network.
        
### 2.1 References

[1] X.-W. Yang, H. Steck, and Y. Liu, “Circle-based recommendation in online social networks,” in Proc. 18th ACM SIGKDD Int. Conf. KDD, New York, NY, USA, Aug. 2012, pp. 1267–1275.

[2] M. Jiang et al., “Social contextual recommendation,” in Proc. 21st ACM Int. CIKM, New York, NY, USA, 2012, pp. 45–54.

[3] H. Feng and X. Qian, “Recommendation via user’s personality and social contextual,” in Proc. 22nd ACM CIKM, New York, NY, USA, 2013

[4] R. Sinha and K. Swearingen, “Comparing recommendations made by online systems and friends,” in Proc. DELOS Workshop Personalisation Recommender Systems Digital Libraries, Dublin, Ireland, 2001.

# 3. Approach / Methods

### 3.1 Dataset Compression
        


### 3.2 Building Matrices
        


### 3.3 Training using gradient descent  - Learning the latent features
        

# 4. Evaluation

      

# 5. Conclusion and Next Steps

        